# 1. Overview

The notebook covers data preparation for ASML and its suppliers, focusing on combining weekly stock data and adding binary variables for exogenous events like COVID, geopolitical tensions, and tech regulations. It starts with data loading and merging steps, ensuring a consistent weekly frequency across datasets. After checking for missing values, the document introduces binary indicators to represent the occurrence of specific external events over time. Finally, the combined dataset is exported for further analysis, encapsulating all relevant market and geopolitical factors.

# 2. Data Understanding

## 2.1. Data Description

## 2.2. Import Necessary Libraries

In [1]:
import pandas as pd

## 2.3. Define global variables

In [2]:
input_data_asml = 'data/weekly_data_asml.csv'
input_data_suppliers = 'data/weekly_data_suppliers.csv'
output_data = 'data/weekly_data_combination.csv'

## 2.4. Functions

In [3]:
# Function to add exogenous variables to the future_dataframe
def add_exogenous_variables(weekly_data):
    """
    Input: future_df (DataFrame) - A DataFrame containing future dates in the 'ds' column.
    Output: future_df (DataFrame) - The same DataFrame with added binary columns for each exogenous event.
    Logic: Adds binary indicators for each specified geopolitical and economic event based on predefined date ranges.
    """
    
    # Define the date ranges for each exogenous event
    covid_start = '2020-01-01'
    covid_end = '2022-12-31'

    geopolitical_tension_start = '2022-02-01'
    geopolitical_tension_end = '2025-12-31'

    trade_sanctions_start = '2018-07-01'
    trade_sanctions_end = '2029-12-31'

    tech_regulation_start = '2020-06-01'
    tech_regulation_end = '2029-12-31'

    new_product_launch_start = '2023-12-01'
    new_product_launch_end = '2025-12-31'

    israel_gaza_conflict_start = '2023-10-07'
    israel_gaza_conflict_end = '2025-12-31'

    # COVID Period
    weekly_data['COVID_Period'] = ((weekly_data.index >= covid_start) & 
                                 (weekly_data.index <= covid_end)).astype(int)
    
    # Geopolitical Tensions (Ukraine Conflict)
    weekly_data['Geopolitical_Tension'] = ((weekly_data.index >= geopolitical_tension_start) & 
                                         (weekly_data.index <= geopolitical_tension_end)).astype(int)
    
    # US-China Trade War
    weekly_data['Trade_Sanctions'] = ((weekly_data.index >= trade_sanctions_start) & 
                                    (weekly_data.index <= trade_sanctions_end)).astype(int)
    
    # Tech Regulation (Export Restrictions)
    weekly_data['Tech_Regulation'] = ((weekly_data.index >= tech_regulation_start) & 
                                    (weekly_data.index <= tech_regulation_end)).astype(int)
    
    # New EUV Machine Launch (TWINSCAN EXE:5000)
    weekly_data['New_Product_Launch'] = ((weekly_data.index >= new_product_launch_start) & 
                                       (weekly_data.index <= new_product_launch_end)).astype(int)
    
    # Israel-Gaza Conflict
    weekly_data['Israel_Gaza_Conflict'] = ((weekly_data.index >= israel_gaza_conflict_start) & 
                                         (weekly_data.index <= israel_gaza_conflict_end)).astype(int)

## 2.5. Code

### 2.5.1. Looking at the datasets

**ASML**

In [4]:
weekly_data_asml = pd.read_csv(input_data_asml, sep=';')

# Ensure the 'Date' column is in datetime format
weekly_data_asml['Date'] = pd.to_datetime(weekly_data_asml['Date'])

# Set the 'Date' column as the index to make it a DatetimeIndex
weekly_data_asml.set_index('Date', inplace=True)

# Explicitly set the frequency of the Date index
weekly_data_asml = weekly_data_asml.asfreq('W')

# Rename the 'Close' column to 'Close_ASML'
weekly_data_asml.rename(columns={'Close': 'Close_ASML'}, inplace=True)

weekly_data_asml

,Close_ASML
Date,
2018-01-14,168.662354
2018-01-21,185.736542
2018-01-28,191.564713
2018-02-04,187.825378
2018-02-11,174.080200
...,...
2024-10-27,710.037903
2024-11-03,683.830017
2024-11-10,671.159973


**Suppliers**

In [5]:
weekly_data_suppliers = pd.read_csv(input_data_suppliers, sep=';')

# Ensure the 'Date' column is in datetime format
weekly_data_suppliers['Date'] = pd.to_datetime(weekly_data_suppliers['Date'])

# Set the 'Date' column as the index to make it a DatetimeIndex
weekly_data_suppliers.set_index('Date', inplace=True)

# Explicitly set the frequency of the Date index
weekly_data_suppliers = weekly_data_suppliers.asfreq('W')

# Drop certain columns
# weekly_data_suppliers = weekly_data_suppliers.drop(['Date.1'], axis=1)

weekly_data_suppliers

,Close_Carl_Zeiss_Meditec,Close_Lam_Research_Corporation,Close_Tokyo_Electron_Limited
Date,,,
2018-01-07,59.808327,10.975054,31.770000
2018-01-14,60.946632,10.902242,32.766666
2018-01-21,60.851768,11.676138,34.386667
2018-01-28,61.373486,11.899403,34.026669
2018-02-04,61.335552,10.869248,31.420000
...,...,...,...
2024-10-27,67.529999,73.000000,76.709999
2024-11-03,65.550003,75.449997,77.500000
2024-11-10,62.200001,76.779999,76.449997


Let's create a weekly_data dataframe from both weekly_data_asml and weekly_data_suppliers

In [6]:
weekly_data = weekly_data_asml.join(weekly_data_suppliers, how='inner')
weekly_data

,Close_ASML,Close_Carl_Zeiss_Meditec,Close_Lam_Research_Corporation,Close_Tokyo_Electron_Limited
Date,,,,
2018-01-14,168.662354,60.946632,10.902242,32.766666
2018-01-21,185.736542,60.851768,11.676138,34.386667
2018-01-28,191.564713,61.373486,11.899403,34.026669
2018-02-04,187.825378,61.335552,10.869248,31.420000
2018-02-11,174.080200,58.831280,9.690061,28.526667
...,...,...,...,...
2024-10-27,710.037903,67.529999,73.000000,76.709999
2024-11-03,683.830017,65.550003,75.449997,77.500000
2024-11-10,671.159973,62.200001,76.779999,76.449997


Display the total number of nulls per column


In [7]:
nulls_per_column = weekly_data.isnull().sum()

nulls_per_column

Close_ASML                        0
Close_Carl_Zeiss_Meditec          0
Close_Lam_Research_Corporation    0
Close_Tokyo_Electron_Limited      0
dtype: int64

### 2.5.2 Add binary variables

In [8]:
# Apply the function to weekly_data
add_exogenous_variables(weekly_data)

In [9]:
weekly_data

,Close_ASML,Close_Carl_Zeiss_Meditec,Close_Lam_Research_Corporation,Close_Tokyo_Electron_Limited,COVID_Period,Geopolitical_Tension,Trade_Sanctions,Tech_Regulation,New_Product_Launch,Israel_Gaza_Conflict
Date,,,,,,,,,,
2018-01-14,168.662354,60.946632,10.902242,32.766666,0,0,0,0,0,0
2018-01-21,185.736542,60.851768,11.676138,34.386667,0,0,0,0,0,0
2018-01-28,191.564713,61.373486,11.899403,34.026669,0,0,0,0,0,0
2018-02-04,187.825378,61.335552,10.869248,31.420000,0,0,0,0,0,0
2018-02-11,174.080200,58.831280,9.690061,28.526667,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2024-10-27,710.037903,67.529999,73.000000,76.709999,0,1,1,1,1,1
2024-11-03,683.830017,65.550003,75.449997,77.500000,0,1,1,1,1,1
2024-11-10,671.159973,62.200001,76.779999,76.449997,0,1,1,1,1,1


# 3. Export

In [10]:
# Export the weekly_data_suppliers to a CSV file
weekly_data.to_csv(output_data, index=True, sep=';')